In [1]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
from transformers import ResNetForImageClassification
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn import Sequential, Flatten
from sklearn.svm import NuSVC, SVC
import torch

In [2]:
model_checkpoint = 'microsoft/resnet-50'
model = ResNetForImageClassification.from_pretrained(model_checkpoint)

model.classifier = Sequential(
    Flatten(start_dim=1, end_dim=-1)
)

In [3]:
orb = cv.ORB_create(64)

In [4]:
folder_path = "C:/Users/thedh/OneDrive/Desktop/Research/Combine Test 1/data"

In [5]:
features = []
labels = []
handcrafted_features = []
deep_features = []

In [6]:
def extract_label_from_image_name(image_name):
    # Assuming the label is encoded in the last 9 characters
    label = image_name[-14:-5]
    return label


In [ ]:
for image_name in os.listdir(folder_path):
    # Assuming image names have a code that corresponds to the label
    label = extract_label_from_image_name(image_name)

    # Load and preprocess the image
    image_path = os.path.join(folder_path, image_name)
    img = cv.imread(image_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # ORB feature extraction
    kp = orb.detect(img, None)
    if kp:
        kp, des = orb.compute(img, kp)
        handcrafted_features.append(des.flatten())

        # Check if descriptors are not None and not empty
        if des is not None and des.size > 0:
            # ResNet-50 feature extraction
            image = Image.open(image_path).resize((224, 224))
            if image.mode != 'RGB':
                image = image.convert('RGB')
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = transform(image)
            inputs = image.unsqueeze(0)
            outputs = model(inputs)
            logits = outputs.logits
            resnet_features = logits.detach().numpy().flatten()
            deep_features.append(resnet_features)

            # Check if resnet_features is not empty
            if resnet_features.size > 0:
                # Combine features
                combined_features = np.concatenate((resnet_features, des.flatten()))

                # Append to lists
                features.append(combined_features)
                labels.append(label)

In [13]:
for i in range(len(handcrafted_features)):
    if type(handcrafted_features[i]) != list:
        handcrafted_features[i] = handcrafted_features[i].tolist()
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))
        handcrafted_features[i] = handcrafted_features[i] + [0]*(2048 - len(handcrafted_features[i]))

for i in range(len(handcrafted_features)):
    if len(handcrafted_features[i]) != 2048:
        print(len(handcrafted_features[i]))


handcrafted_features = np.array(handcrafted_features)
deep_features = np.array(deep_features)

print(handcrafted_features.shape)
print(deep_features.shape)

combined_features = np.stack((deep_features, handcrafted_features), axis=1)
print(combined_features.shape)

1248
352
512
1984
1728
1632
1536
1984
1984
1952
1984
1088
2016
1984
1760
2016
640
1152
1824
2016
1824
1248
960
1888
1984
1920
1664
1952
1984
1632
1824
1888
1984
1664
1984
1824
2016
1728
1984
2016
1920
992
1856
1920
1856
1984
608
1440
1984
1952
2016
2016
1984
1824
2016
1984
768
1696
1920
1216
1984
1824
2016
2016
1280
2016
1984
1600
2016
960
832
1536
1504
1472
256
1984
1408
1568
1824
1824
1984
832
1632
1536
1984
1600
1632
1984
1152
1824
832
1984
2016
1632
1792
1984
1792
1984
1408
1984
2016
1824
480
1664
288
1824
992
1632
1824
1664
1376
1440
864
1984
1856
1856
1536
1984
224
1952
1984
2016
1888
1984
1888
832
1088
1984
1888
1760
1056
2016
1472
2016
1952
1952
1152
1824
1344
1216
2016
672
1824
1920
1248
1984
1568
800
1792
1632
1824
1952
1248
864
2016
1152
2016
1408
2016
2016
608
1952
1920
1728
1856
1984
1824
1824
1920
1824
1632
1856
1984
1952
608
1984
1888
1728
1984
1792
768
1952
1440
1984
2016
1984
1312
1568
1984
1856
1760
2016
1664
1920
1888
672
1952
1632
1984
1984
1984
1504
2016
1824
800
5

In [15]:
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.1, random_state=100)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train = np.array(X_train).reshape((X_train.shape[0], 4096))
X_test = np.array(X_test).reshape((X_test.shape[0], 4096))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(44983, 2, 2048) (44983,) (4999, 2, 2048) (4999,)
(44983, 4096) (44983,) (4999, 4096) (4999,)


In [16]:
svc = SVC()

svc.fit(X_train, y_train)

SVC()

In [ ]:
preds = svc.predict(X_test)

ct=0
tot=0
for i in range(len(preds)):
    tot+=1
    if preds[i] == y_test[i]:
        ct+=1
        
print(f'Accuracy is {ct}/{tot} = {ct/tot}')